In [ ]:
import pandas as pd
import time
import datetime
import requests
import numpy as np

In [11]:

def read_ticker(ticker):
    period1 = int(time.mktime(datetime.datetime(2000, 1, 1, 0, 0).timetuple()))
    period2 = int(time.mktime(datetime.datetime(2023, 6, 26, 23, 59).timetuple()))
    interval = '1d' # 1d, 1m

    query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history'
    df = pd.read_csv(query_string, index_col=0, parse_dates=True)
    return df


In [14]:
NASDAQ100_stocklist = [
"AAPL",
"ILMN",
"DLTR",
"HON",
"FTNT",
"CSGP",
"INTU",
"PCAR",
"ATVI",
"MDLZ",
"AMGN",
"MELI",
"ZM",
"CEG",
"BKR",
"META",
"MCHP",
"ZS",
"VRSK",
"MRNA",
"AMAT",
"SIRI",
"MRVL",
"KDP",
"ENPH",
"MAR",
"AZN",
"SBUX",
"TSLA",
"JD",
'NDX',
]

df_list = {}

for ticker in NASDAQ100_stocklist:
    print(ticker)
    df_list[ticker] = read_ticker(ticker)

AAPL
ILMN
DLTR
HON
FTNT
CSGP
INTU
PCAR
ATVI
MDLZ
AMGN
MELI
ZM
CEG
BKR
META
MCHP
ZS
VRSK
MRNA
AMAT
SIRI
MRVL
KDP
ENPH
MAR
AZN
SBUX
TSLA
JD
NDX


In [15]:
for ticker, stock_df in df_list.items():
    print('ticker: {} -> {}'.format(ticker, stock_df.index.min()))

ticker: AAPL -> 1999-12-31 00:00:00
ticker: ILMN -> 2000-07-28 00:00:00
ticker: DLTR -> 1999-12-31 00:00:00
ticker: HON -> 1999-12-31 00:00:00
ticker: FTNT -> 2009-11-18 00:00:00
ticker: CSGP -> 1999-12-31 00:00:00
ticker: INTU -> 1999-12-31 00:00:00
ticker: PCAR -> 1999-12-31 00:00:00
ticker: ATVI -> 1999-12-31 00:00:00
ticker: MDLZ -> 2001-06-13 00:00:00
ticker: AMGN -> 1999-12-31 00:00:00
ticker: MELI -> 2007-08-10 00:00:00
ticker: ZM -> 2019-04-18 00:00:00
ticker: CEG -> 2022-01-19 00:00:00
ticker: BKR -> 1999-12-31 00:00:00
ticker: META -> 2012-05-18 00:00:00
ticker: MCHP -> 1999-12-31 00:00:00
ticker: ZS -> 2018-03-16 00:00:00
ticker: VRSK -> 2009-10-07 00:00:00
ticker: MRNA -> 2018-12-07 00:00:00
ticker: AMAT -> 1999-12-31 00:00:00
ticker: SIRI -> 1999-12-31 00:00:00
ticker: MRVL -> 2000-06-30 00:00:00
ticker: KDP -> 2008-05-07 00:00:00
ticker: ENPH -> 2012-03-30 00:00:00
ticker: MAR -> 1999-12-31 00:00:00
ticker: AZN -> 1999-12-31 00:00:00
ticker: SBUX -> 1999-12-31 00:00:00
ti

In [19]:
min_date = max(list([stock_df.index.min() for stock_df in df_list.values()]))
max_date = min(list([stock_df.index.max() for stock_df in df_list.values()]))

In [20]:
min_date

Timestamp('2022-01-19 00:00:00')

In [21]:
key_list = [k for k in df_list.keys()]
for key in key_list:
    current_df = df_list[key]
    print(f'Ticker {key}: {current_df.index.min()}')
    if current_df.index.min().year >= 2013:
        del df_list[key]
        print(f'Deleted Ticker {key}')

Ticker AAPL: 1999-12-31 00:00:00
Ticker ILMN: 2000-07-28 00:00:00
Ticker DLTR: 1999-12-31 00:00:00
Ticker HON: 1999-12-31 00:00:00
Ticker FTNT: 2009-11-18 00:00:00
Ticker CSGP: 1999-12-31 00:00:00
Ticker INTU: 1999-12-31 00:00:00
Ticker PCAR: 1999-12-31 00:00:00
Ticker ATVI: 1999-12-31 00:00:00
Ticker MDLZ: 2001-06-13 00:00:00
Ticker AMGN: 1999-12-31 00:00:00
Ticker MELI: 2007-08-10 00:00:00
Ticker ZM: 2019-04-18 00:00:00
Deleted Ticker ZM
Ticker CEG: 2022-01-19 00:00:00
Deleted Ticker CEG
Ticker BKR: 1999-12-31 00:00:00
Ticker META: 2012-05-18 00:00:00
Ticker MCHP: 1999-12-31 00:00:00
Ticker ZS: 2018-03-16 00:00:00
Deleted Ticker ZS
Ticker VRSK: 2009-10-07 00:00:00
Ticker MRNA: 2018-12-07 00:00:00
Deleted Ticker MRNA
Ticker AMAT: 1999-12-31 00:00:00
Ticker SIRI: 1999-12-31 00:00:00
Ticker MRVL: 2000-06-30 00:00:00
Ticker KDP: 2008-05-07 00:00:00
Ticker ENPH: 2012-03-30 00:00:00
Ticker MAR: 1999-12-31 00:00:00
Ticker AZN: 1999-12-31 00:00:00
Ticker SBUX: 1999-12-31 00:00:00
Ticker TSLA

In [23]:
index = pd.date_range(start=min_date, end=max_date, freq='D')
stock_price_df = pd.DataFrame(index=index)
for key, ticker_df in df_list.items():
    stock_price_df = pd.concat([stock_price_df, ticker_df[['Close']]], axis=1)
    stock_price_df.rename(columns={'Close': key}, inplace=True)

In [24]:
stock_price_df

,AAPL,ILMN,DLTR,HON,FTNT,CSGP,INTU,PCAR,ATVI,MDLZ,...,AMAT,SIRI,MRVL,KDP,ENPH,MAR,AZN,SBUX,TSLA,NDX
1999-12-31,0.917969,NaN,10.763889,54.997505,NaN,3.587500,29.968750,5.835391,1.276042,NaN,...,31.671875,44.5000,NaN,NaN,NaN,14.711524,20.875000,3.031250,NaN,3707.830078
2000-01-03,0.999442,NaN,10.861111,54.044136,NaN,3.137500,30.125000,5.728395,1.369792,NaN,...,31.625000,41.1250,NaN,NaN,NaN,14.216284,20.562500,3.082031,NaN,3790.550049
2000-01-04,0.915179,NaN,10.847222,53.090767,NaN,3.237500,30.437500,5.465021,1.328125,NaN,...,30.062500,38.3750,NaN,NaN,NaN,14.128889,19.875000,2.984375,NaN,3546.199951
2000-01-05,0.928571,NaN,10.861111,52.375736,NaN,3.143750,39.218750,5.687243,1.333333,NaN,...,28.937500,40.3125,NaN,NaN,NaN,14.361943,19.968750,3.023438,NaN,3507.310059
2000-01-06,0.848214,NaN,11.055556,53.388695,NaN,3.100000,35.000000,5.613169,1.307292,NaN,...,28.984375,41.5000,NaN,NaN,NaN,14.565865,19.906250,3.132813,NaN,3340.810059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-20,185.009995,202.130005,136.539993,200.399994,71.660004,85.309998,455.130005,77.769997,81.889999,73.260002,...,138.520004,3.7500,60.830002,31.93,171.990005,173.789993,73.949997,101.269997,274.450012,15070.150391
2023-06-21,183.960007,199.869995,142.770004,201.800003,71.110001,87.580002,450.529999,78.519997,80.940002,73.139999,...,136.669998,3.6600,58.849998,31.74,167.990005,176.119995,73.379997,101.870003,259.459991,14867.450195
2023-06-22,187.000000,202.149994,144.070007,200.649994,71.860001,86.400002,450.839996,78.110001,82.300003,73.739998,...,138.919998,3.7500,59.119999,31.92,162.529999,175.690002,73.949997,100.849998,264.609985,15042.320313


In [25]:
stock_price_df.isna().sum()

AAPL     162
ILMN     307
DLTR     162
HON      162
FTNT    2648
CSGP     162
INTU     162
PCAR     162
ATVI     162
MDLZ     527
AMGN     162
MELI    2074
BKR      162
META    3277
MCHP     162
VRSK    2618
AMAT     162
SIRI     162
MRVL     288
KDP     2260
ENPH    3243
MAR      162
AZN      162
SBUX     162
TSLA    2800
NDX      162
dtype: int64

In [26]:
stock_price_df.dropna(inplace=True)

In [29]:
stock_price_df.to_csv('data/NASDAQ100.csv')